In [1]:
# 라이브러리 임포트

import numpy as np
import pandas as pd

import re
import folium
import matplotlib.pyplot as plt
import seaborn as sns

import requests
import json

import warnings
warnings.filterwarnings(action = 'ignore')

import time

from collections import Counter

In [2]:
# 서울 행정구역 json raw파일 (githubcontent)
r = requests.get('https://raw.githubusercontent.com/southkorea/seoul-maps/master/kostat/2013/json/seoul_municipalities_geo_simple.json')
c = r.content
seoul_geo = json.loads(c)

In [3]:
# 강서구만 따로 가져옴

gangseo = '{"type":"Feature","properties":{"code":"11160","name":"강서구","name_eng":"Gangseo-gu","base_year":"2013"},"geometry":{"type":"Polygon","coordinates":[[[126.85984199399667,37.571847855292745],[126.85950389772532,37.5682771531189],[126.8604929702619,37.566825621733905],[126.86837407967381,37.56309561411933],[126.87997924964915,37.55510740490174],[126.89184663862764,37.547373974997114],[126.88736718003831,37.54350482420959],[126.88825757860099,37.54079733630232],[126.8872486543506,37.54079100234749],[126.88280520161935,37.545072121233225],[126.8761093656006,37.54412882794802],[126.8727924099192,37.544853699294116],[126.86637464321238,37.54859191094823],[126.86426635332332,37.54172958759955],[126.86582129720519,37.53817151116851],[126.8655037497872,37.53382926555605],[126.86610073476395,37.52699964144669],[126.85098397861944,37.525098716169985],[126.84257291943153,37.52373707805596],[126.836555914069,37.53367208325903],[126.83716591765655,37.534935320492906],[126.83522688458329,37.539042988809484],[126.8325348945036,37.538953250433295],[126.83191667070415,37.54146500511403],[126.83017074557299,37.542611079015344],[126.82889818288362,37.53909381305992],[126.8242331426722,37.53788078753248],[126.81674221631081,37.5378396500627],[126.81246052552456,37.538810793377344],[126.81143604908785,37.54033621168525],[126.80542840499083,37.54008921830378],[126.80185404828612,37.537645443377826],[126.80090914554204,37.53503009075454],[126.79688612254975,37.53302974862096],[126.79582133969424,37.536641561833754],[126.79693641290046,37.53865858611534],[126.79388711477147,37.53902211451394],[126.7938616681597,37.54104361952839],[126.79075533377627,37.54165251407983],[126.78880225462409,37.54353706379955],[126.78198339188025,37.543449601019624],[126.77756215424237,37.54611355396897],[126.77324417717703,37.5459123450554],[126.76977011413412,37.55052082471595],[126.76700465024426,37.552821566629916],[126.77074629769308,37.55296836994276],[126.77145103135192,37.55434307460708],[126.77879087345151,37.55919525318415],[126.77889121370164,37.5613614424496],[126.77671213061004,37.5645429268672],[126.77986476402239,37.564245932540665],[126.78252024622797,37.565367145342954],[126.78471963959866,37.56745669198498],[126.78398460461828,37.56905814599349],[126.78496836516075,37.57090748246567],[126.79172648531066,37.57472630536462],[126.79539987549317,37.57451148875729],[126.79523455851671,37.57760277954844],[126.7958606942207,37.58019957877273],[126.79799133400897,37.58036436587069],[126.80091228188235,37.5854309825683],[126.80289369340177,37.58621464221784],[126.80131704756816,37.58839794302751],[126.80149834313248,37.59012749570681],[126.7996634054858,37.59296530943065],[126.79910601240701,37.59569886491464],[126.80046544382346,37.59827267924192],[126.80198459129242,37.598541940075755],[126.80268446118524,37.601312560472834],[126.80393696882469,37.601857300987895],[126.80759006979085,37.60089755124775],[126.81814502537962,37.591566052513244],[126.82251438477105,37.5880430810082],[126.82891304761237,37.5855611764797],[126.85302823436479,37.57282468882299],[126.85984199399667,37.571847855292745]]]}}'
gangseo_geo = json.loads(gangseo)

In [4]:
m = folium.Map(
    location = [37.559819, 126.825895],
    zoom_start = 12.5, 
)

folium.GeoJson(
    gangseo_geo,
    name='지역구'
).add_to(m)

# folium.Marker([37.55,126.84],
#               popup="(5,3)",
#               tooltip="(5,3)").add_to(m)

# m.add_child(folium.LatLngPopup())

m

In [5]:
# 강서구 위경도 min/max

maxlat = np.array(gangseo_geo['geometry']['coordinates'])[0][:,1].max()
minlat = np.array(gangseo_geo['geometry']['coordinates'])[0][:,1].min()

maxlon = np.array(gangseo_geo['geometry']['coordinates'])[0][:,0].max()
minlon = np.array(gangseo_geo['geometry']['coordinates'])[0][:,0].min()

print(minlat, maxlat, minlon, maxlon)

37.52373707805596 37.601857300987895 126.76700465024426 126.89184663862764


In [6]:
# 최종 25 * 25로 스팟 나누기

maxlat = 37.601857300987895
minlat = 37.52373707805596

maxlon = 126.89184663862764
minlon = 126.76700465024426

lat_spot = [minlat]
for i in range(1,26):
    lat_spot.append(minlat + (i * ((maxlat - minlat) / 25)))

lon_spot = [minlon]
for i in range(1,26):
    lon_spot.append(minlon + (i * ((maxlon - minlon) / 25)))

In [7]:
def lonlat(lat, lon):
    for i in range(25):
        if (lat_spot[i] <= lat) & (lat < lat_spot[i+1]):
            lat_label = 'lat{}'.format(i)    
    for i in range(25):
        if (lon_spot[i] <= lon) & (lon < lon_spot[i+1]):
            lon_label = 'lon{}'.format(i)   
    return lon_label + '_' + lat_label # 위치 파악의 용이성 때문에 longitude를 앞으로 배치

In [8]:
# example

lonlat(37.55,126.84)

'lon14_lat8'

In [9]:
# 스팟 시각화로 확인

lat_start = [[maxlat,i] for i in lon_spot]
lat_end = [[minlat,i] for i in lon_spot]

lon_start = [[i, maxlon] for i in lat_spot]
lon_end = [[i, minlon] for i in lat_spot]

m = folium.Map(
    location = [37.559819, 126.825895],
    zoom_start = 12.5, 
)

for i in range(26): 
    folium.PolyLine(locations = [lat_start[i], lat_end[i]], tooltip = 'PolyLine', color = 'black', weight = 1).add_to(m)
    folium.PolyLine(locations = [lon_start[i], lon_end[i]], tooltip = 'PolyLine', color = 'black', weight = 1).add_to(m)

folium.GeoJson(
    gangseo_geo,
    name='지역구'
).add_to(m)

m

In [13]:
# import os

# alldf = []
# for filename in os.listdir("위경도데이터"):
#     with open(os.path.join("위경도데이터", filename), 'r') as f:
#         text = f.read()
#         alldf.append(text)

In [14]:
# 데이터 불러오기

dir = '위경도데이터/'

afterschoolacademy = pd.read_csv(dir + 'afterschoolacademy.csv', encoding = 'cp949')
busstation = pd.read_csv(dir + 'busstation.csv', encoding = 'cp949')
cctv = pd.read_csv(dir + 'cctv.csv', encoding = 'cp949')
childcarelibrary = pd.read_csv(dir + 'childcarelibrary.csv', encoding = 'cp949')
childrencenter = pd.read_csv(dir + 'childrencenter.csv', encoding = 'cp949')
crosswalk = pd.read_csv(dir + 'crosswalk.csv', encoding = 'cp949')
elementary = pd.read_csv(dir + 'elementary.csv', encoding = 'cp949')
emergencyroom = pd.read_csv(dir + 'emergencyroom.csv', encoding = 'utf-8')
female = pd.read_csv(dir + 'female.csv', encoding = 'cp949')
firestation = pd.read_csv(dir + 'firestation.csv', encoding = 'cp949')
high = pd.read_csv(dir + 'high.csv', encoding = 'cp949')
hospital = pd.read_csv(dir + 'hospital.csv', encoding = 'utf-8')
kindergarden = pd.read_csv(dir + 'kindergarden.csv', encoding = 'cp949')
library = pd.read_csv(dir + 'library.csv', encoding = 'cp949')
lifelongedu = pd.read_csv(dir + 'lifelongedu.csv', encoding = 'cp949')
middleschool = pd.read_csv(dir + 'middleschool.csv', encoding = 'cp949')
parentingroom = pd.read_csv(dir + 'parentingroom.csv', encoding = 'cp949')
park = pd.read_csv(dir + 'park.csv', encoding = 'cp949')
police = pd.read_csv(dir + 'police.csv', encoding = 'cp949')
publicparking = pd.read_csv(dir + 'publicparking.csv', encoding = 'cp949')
safetycenter = pd.read_csv(dir + 'safetycenter.csv', encoding = 'cp949')
seniorcenter = pd.read_csv(dir + 'seniorcenter.csv', encoding = 'cp949')
seniorwelfare = pd.read_csv(dir + 'seniorwelfare.csv', encoding = 'cp949')
subway = pd.read_csv(dir + 'subway.csv', encoding = 'cp949')
taxistop = pd.read_csv(dir + 'taxistop.csv', encoding = 'cp949')
trashcan = pd.read_csv(dir + 'trashcan.csv', encoding = 'cp949')

In [15]:
# 가까운 애들 찾기

def nearspot(input_, n): # n에 홀수만 넣기!
    list_ = []
    lon_spot = int(re.findall(r'\d+', input_)[0]) # lon
    lat_spot = int(re.findall(r'\d+', input_)[1]) # lat
    for i in [i for i in list(range(lon_spot - (n//2), lon_spot + (n//2) + 1)) if i >= 0]:
        for j in [j for j in list(range(lat_spot - (n//2), lat_spot + (n//2) + 1)) if j >= 0]:
            list_.append('lon' + str(i) + '_' + 'lat' + str(j))
    return list_

In [16]:
# example

nearspot('lon23_lat3', 3) # (lon,lat)을 행렬로 봤을 때 기준값 주위 3*3 출력

['lon22_lat2',
 'lon22_lat3',
 'lon22_lat4',
 'lon23_lat2',
 'lon23_lat3',
 'lon23_lat4',
 'lon24_lat2',
 'lon24_lat3',
 'lon24_lat4']

In [17]:
subwaytoconcat = subway[['type','latitude','longitude']] # 지하철도 역이름 제외하고 담기

In [18]:
# 모든 데이터프레임 리스트로

df_list = [afterschoolacademy, busstation, cctv, childcarelibrary, childrencenter,
crosswalk, elementary, emergencyroom, female, firestation,
high, hospital, kindergarden, library, lifelongedu,
middleschool, parentingroom, park, police, publicparking,
safetycenter, seniorcenter, seniorwelfare, taxistop, trashcan, subwaytoconcat]

In [19]:
# 합치기 + 결측치: 수집 도중 추가된 의미없는 데이터 → 삭제

concat_df = pd.concat(df_list).dropna().reset_index(drop = True)

In [20]:
# 결측치 없음

concat_df.isnull().sum()

type         0
latitude     0
longitude    0
dtype: int64

In [21]:
# 데이터 타입 확인 - longitude dtype = object: 숫자가 아닌 값이 포함되어 있음

concat_df.dtypes

type          object
latitude     float64
longitude     object
dtype: object

In [39]:
# 이상치 발견

a = []
for i in concat_df['longitude']:
    try: 
        a.append(float(i))
    except: 
        a.append(i + ' check')
        
[i for i in a if str(i).find(' check!!') != -1] # 이상치 하나 발견

127/02437 check!!


In [23]:
# 한 개의 이상치 바꾸기

concat_df.loc[concat_df['longitude'] == ('127/02437'),'longitude'] = 127.02437

In [24]:
# longitude dtype 변경(object → float64

concat_df['longitude'] = concat_df['longitude'].astype('float64')

In [25]:
concat_df.reset_index(drop = True, inplace = True)

In [26]:
concat_df.dtypes

type          object
latitude     float64
longitude    float64
dtype: object

In [27]:
concat_df.describe()

,latitude,longitude
count,8907.000000,8907.000000
mean,37.551693,126.844939
std,0.039108,0.046018
min,35.566191,126.484412
25%,37.540484,126.826587
50%,37.552975,126.841990
75%,37.563348,126.853267
max,37.875769,129.221731


In [29]:
# 중복 싸그리 제거

for i in df_list:
    i.drop_duplicates(inplace = True)

In [30]:
# 인덱스 재설정

for i in df_list:
    i.reset_index(inplace = True, drop = True)

In [31]:
# 다시 합치기

concat_df = pd.concat(df_list)

# 한 개의 이상치 바꾸기 (위의 코드 가져옴)

concat_df.loc[concat_df['longitude'] == ('127/02437'),'longitude'] = 127.02437
concat_df['longitude'] = concat_df['longitude'].astype('float')
concat_df.reset_index(drop = True, inplace = True)

In [32]:
concat_df.type.unique()

array(['academy', 'busstation', 'CCTV', 'childcarelibrary',
       'childrencenter', 'crosswalk', 'elementary', 'emergencyroom',
       'femalezone', 'firestation', 'high', 'hospital', 'kindergarden',
       'library', 'lifelongedu', 'middle', 'infant', 'park', 'police',
       'publicparking', 'chiancenter', 'seniorcenter', 'oldwelfare',
       'taxistop', 'trashcan', 'subway'], dtype=object)

In [33]:
concat_df.shape

(5050, 3)

In [34]:
# 강서구 최대최소값 안에 없는 값들: 강서구 데이터 아니라고 판단: 다 삭제

concat_df = concat_df.loc[((concat_df['latitude'] >= minlat) & (concat_df['latitude'] <= maxlat) &
                           (concat_df['longitude'] >= minlon) & (concat_df['longitude'] <= maxlon))].reset_index(drop = True)

In [35]:
# 중심 주위 몇*몇으로 할지
howmuchnumber = [3, 1, 1, 3, 
                 1, 1, 3, 5,
                 1, 5, 5, 1, 1,
                 3, 5, 5, 3, 5, 5,
                 3, 3, 1, 5,
                 1, 1, 5]

In [36]:
# 강서구인 위치만 추출한 데이터

final_df = pd.read_csv('지도 데이터.csv')
final_df.head(2) # location = all: 중심지역(100% 강서구)/ side = 외곽지역

,lonlat,location
0,lon15_lat1,all
1,lon16_lat1,all


In [37]:
for whattype,how in zip(concat_df['type'].unique(), howmuchnumber):

    temp_df = concat_df.loc[concat_df['type'] == whattype]
    
    all_nearspot_list = []
    for lat,lon in zip(temp_df['latitude'], temp_df['longitude']):
        all_nearspot_list.extend(nearspot(lonlat(lat,lon), how))
    
    dftomerge = pd.DataFrame({'lonlat': Counter(all_nearspot_list).keys(), whattype: Counter(all_nearspot_list).values()})
    final_df = pd.merge(final_df,
             dftomerge,
             on = 'lonlat',
             how = 'left').fillna(0)

In [38]:
# final_df.to_csv('final_dataset.csv', index = False)